Mumford-Shah image segmentation
===============================

This code provides a CPU (slow\*) implementation of an approximation to Mumford-Shah image segmentation. 

Based on:
 * "A first-order primal-dual algorithm for convex problems with applications to imaging" 
Chambolle, Antonin and Pock, Thomas (2011)
Journal of Mathematical Imaging and Vision. 40(1)

Intersection of convex set projection method based on 
 * "A cyclic projection algorithm via duality"
Gaffke, Norbert and Mathar, Rudolf (1989)
Metrika. 36(1)

Unit simplex projection based on 
 * "Projection onto the probability simplex : An efficient algorithm with a
 simple proof and an application"
Wang, Weiran and Miguel, A (2013)
arXiv:1309.1541v1

\* GPU implementation coming... 

![title](butterfly_demo.png)


What are we doing?
---------------------

The basic idea is to fix a set of $k$ colors we are going to segment the image with. We want to do so in a way that avoids high spatial variability while maintaining sharp transitions between regions of very different color. The solution is to use a total-variation (TV) penalty. Let $\{c_i\}^k$ be our set set of colors and let $g(\mathbf{x})$ be our image. Let $\mathbf{f}$ be the vector of distance between the image and each of our colors:
$$
f_l = \lambda \int |g(x)-c_i|^2\,d\mathbf{x}\qquad l = 1,\dots, k
$$

Let $\mathbf{u}(x)$ be our vector that weights contributions from each color $c_i$ to produce a final pixel intensity at $x$:
$$
I(x) = u(x)^T\mathbf{c}.
$$
Thus $\mathbf{u}$ selects our color. We want the vector $\mathbf{u}$ to be _sparsely_ varying as a function of $x$.

We set this up as the following optimization problem
$$
\min_{u\in U} \frac{1}{2}\sum_{l=1}^k \int_\Omega |Du_l| + \sum_{l=1}^k\int_\Omega u_lf_l\,dx
$$
subject to particular constraints described more below. 

Why TV minimization?
-----------------------

Here $\int_\Omega|Du|$ denotes the total variation of the function $u$, with $Du$ being the _distributional derivative_ of $u$. For sufficiently smooth functions, $u\in W^{1,1}(\Omega)$, it reduces to:
$$
\int_\Omega |Du| = \int_\Omega|\nabla u|\,dx.
$$

There is a nice relation between the total variation regularization and perimeter/partition minimization. This is allowed by the fact that
$$
\int_\Omega |Du| = \int_{-\infty}^{\infty}\text{Per}(\{u>s\},\Omega)\,ds,
$$
that is, that the total variation of $u$ is equivalent to the sum of the surfaces of its level sets. This means minimizing TV is equivalent to minimizing the length of boundaries of contiguous regions of color. 

Relation to image segmentation
----------------------------------

Coming back to our image segmentation problem. Let $R_l$ denote a partition of the domain $\Omega$. Each partition has a color $c_i$. The piece-wise constant, Mumford-Shah problem is then
$$
\min_{(R_l)_{l=1}^k, (c_l)_{l=1}^k}\frac{1}{2}\sum_{l=1}^k \text{Per}(R_l;\Omega) + \frac{\lambda}{2}\sum_{l=1}^k \int_{R_l}|g(x)-c_l|^2\,dx.
$$
The original specification treats the colors $c_l$ as unknown. This is related to a model from statistical physics known as the [Potts model](https://en.wikipedia.org/wiki/Potts_model). However this problem can be shown to be NP-hard. A convex problem can be obtained by treating $c_l$ are known. (An easy way to choose sensible colors is to choose a $k$, and perform k-means clustering to pick the $c_l$s.)

We thus relate the minimal partition problem above to a type of vector total variation minimization:
$$
\min_{(u_l)_{l=1}^k}J(u) + \sum_{l=1}^k \int_\Omega u_lf_l \,dx,\quad u_l(x)\ge 0, \sum_{l=1}^k u_l(x) = 1,\forall x\in \Omega
$$
Where different choices of energy $J(u)$ are possible. The simplest is to choose
$$
J_1(u) = \frac{1}{2}\sum_{k=1}^l \int_\Omega |Du_l|,
$$
and a more principled choice is
$$
J_2(u) = \int_\Omega \Psi(Du),
$$
where
$$
\Psi(Du) = \sup_q\{\langle p,q:|q_l-q_m|\le 1,1\le l<m\le k \rangle \}.
$$

In this implementation we use the simpler functional $J_1(u)$. 

The primal-dual implementation we solve is then:

$$
\min_{u=(u_l)_{l=1}^k}\max_{p=(p_l)_{l=1}^k}\left(\sum_{l=1}^k\langle \nabla u_l, p_l\rangle + \langle u_l,f_l\rangle \right)+\delta_U(u) - \delta_P(p).
$$
Where $U$ is the pixel-wise unit simplex:
$$
U=\left\{u\in X^k:(u_l)_{i,j}\ge 0, \sum_{l=1}^k (u_l)_{i,j} = 1\right\}
$$
and $P$ is the set
$$
P = \left\{ p\in Y^k,\|p_l\|_\infty \le \frac{1}{2} \right\}
$$